pip install pvlib pysolar scipy pandas


pvlib

In [21]:
import pandas as pd
import numpy as np
from pvlib import solarposition
from datetime import datetime, timedelta
from scipy.optimize import minimize_scalar

# Inputs (example)
latitude = 68.35594288
longitude = 19.04520892
date = pd.Timestamp('1988-08-09', tz='UTC') # landsat uses gmt: https://www.usgs.gov/faqs/how-can-i-find-acquisition-time-a-landsat-scene
target_elevation = 36.63043526  # degrees
target_azimuth = 162.30177663   # degrees
# 09:52:34.5440880Z

	
latitude = 68.35594288
longitude = 19.04520892
date = pd.Timestamp('2024-09-05', tz='UTC') # landsat uses gmt: https://www.usgs.gov/faqs/how-can-i-find-acquisition-time-a-landsat-scene
target_elevation = 27.220000  # degrees
target_azimuth = 115.460000 # degrees

# Objective function: minimize the difference in solar position
def objective(mins_since_midnight):
    dt = date + pd.Timedelta(minutes=mins_since_midnight)
    pos = solarposition.get_solarposition(dt, latitude, longitude)
    elev = pos['apparent_elevation'].values[0]
    azim = pos['azimuth'].values[0]
    return (elev - target_elevation)**2 + (azim - target_azimuth)**2

# Optimize over a 24-hour period
res = minimize_scalar(objective, bounds=(0, 1440), method='bounded')
best_time = date + pd.Timedelta(minutes=res.x)

print("Estimated time of day (UTC):", best_time.time())
print("Full datetime (UTC):", best_time)

Estimated time of day (UTC): 06:52:51.847483
Full datetime (UTC): 2024-09-05 06:52:51.847483942+00:00


In [29]:
def calc_time_by_solar(latitude, longitude, date, timezone, target_elevation, target_azimuth):

    # Make sure the variables passed to the function are in the correct format
    date = pd.Timestamp(date, tz = timezone) # add a timezone just in case (probably just "UTC")
    target_elevation = abs(27.220000) # must be positive
    target_azimuth = abs(target_azimuth) # must be positive
    
    # Minimize the difference in solar position
    def objective(mins_since_midnight):
        dt = date + pd.Timedelta(minutes = mins_since_midnight)
        pos = solarposition.get_solarposition(dt, latitude, longitude)
        elev = pos['apparent_elevation'].values[0]
        azim = pos['azimuth'].values[0]
        return (elev - target_elevation)**2 + (azim - target_azimuth)**2
    
    # Optimize over a 24-hour period
    res = minimize_scalar(objective, bounds = (0, 1440), method = 'bounded') # using 1440 for number of mins in 24 hours. Don't need more resolution than this
    recovered_datetime = date + pd.Timedelta(minutes = res.x) # Get the time of day (res.x) and add it to the input date.
    
    return recovered_datetime

#test = calc_time_by_solar(
#    latitude = 68.35594288,
#    longitude = 19.04520892,
#    date = pd.Timestamp('2024-09-05', tz='UTC'),
#    target_elevation = 27.220000,
#    target_azimuth = 115.460000
#    )

#test = calc_time_by_solar(
#    latitude = 68.35594288,
#    longitude = 19.04520892,
#    date = "1988-08-09",
#    timezone = "UTC",
#    target_elevation = 36.63043526, 
#    target_azimuth = 162.30177663
#    )


def convert_azimuth(azimuth):
    # Convert azimuth from -180 to 180 range to 0 to 360 range
    if azimuth < 0:
        azimuth += 360
    return azimuth

# Example usage
solar_azimuths = [-150, -90, 0, 90, 150]  # List of example azimuths
converted_azimuths = [convert_azimuth(az) for az in solar_azimuths]

print(converted_azimuths)


test = calc_time_by_solar(
    latitude = 68.35594288,
    longitude = 19.04520892,
    date = "2002-07-04",
    timezone = "UTC",
    target_elevation = 43.31, 
    target_azimuth = convert_azimuth(-98.93)
    )

print("Estimated time of day (UTC):", test.time())
print("Full datetime (UTC):", test)

[210, 270, 0, 90, 150]
Estimated time of day (UTC): 15:33:12.561827
Full datetime (UTC): 2002-07-04 15:33:12.561827389+00:00


pvlib but with millseconds and elevation (instead of apparent elevation)

In [37]:
import pandas as pd
import numpy as np
from pvlib import solarposition
from scipy.optimize import minimize_scalar

# Inputs
latitude = 68.35594288
longitude = 19.04520892
date = pd.Timestamp('1988-08-09', tz='UTC')
target_elevation = 36.63043526  # degrees
target_azimuth = 162.30177663   # degrees

# Objective function: seconds since midnight
def objective(secs_since_midnight):
    dt = date + pd.Timedelta(seconds=secs_since_midnight)
    pos = solarposition.get_solarposition(dt, latitude, longitude)
    #elev = pos['apparent_elevation'].values[0]
    elev = pos['elevation'].values[0]  # instead of 'apparent_elevation'
    azim = pos['azimuth'].values[0]
    return (elev - target_elevation)**2 + (azim - target_azimuth)**2

# Optimize over a 24-hour period in seconds
res = minimize_scalar(objective, bounds=(0, 86400), method='bounded', options={'xatol': 0.01})

# Best estimated time
best_time = date + pd.Timedelta(seconds=res.x)

print("Estimated time of day (UTC):", best_time.time())
print("Full datetime (UTC):", best_time)


Estimated time of day (UTC): 09:50:29.590540
Full datetime (UTC): 1988-08-09 09:50:29.590540080+00:00


pysolar

In [30]:
from pysolar.solar import get_altitude, get_azimuth
from datetime import datetime, timedelta
from scipy.optimize import minimize_scalar
import pytz

latitude = 68.35594288
longitude = 19.04520892
date = pd.Timestamp('1988-08-09', tz='UTC') # landsat uses gmt: https://www.usgs.gov/faqs/how-can-i-find-acquisition-time-a-landsat-scene
target_elevation = 36.63043526  # degrees
target_azimuth = 162.30177663   # degrees
# 09:52:34.5440880Z

# Objective function
def objective(mins_since_midnight):
    dt = date + timedelta(minutes=mins_since_midnight)
    elev = get_altitude(latitude, longitude, dt)
    azim = get_azimuth(latitude, longitude, dt)
    return (elev - target_elevation)**2 + (azim - target_azimuth)**2

# Optimize over a 24-hour window
res = minimize_scalar(objective, bounds=(0, 1440), method='bounded')
best_time = date + timedelta(minutes=res.x)

print("Estimated time of day (pysolar):", best_time.time())

# should produce: 09:52:34.5440880Z

Estimated time of day (pysolar): 09:50:29.460480


skyfield

In [39]:
from skyfield.api import load, wgs84
from datetime import datetime, timezone

ts = load.timescale()
t = ts.utc(1988, 8, 9, 9, 52, 34.544088)

eph = load('de421.bsp')
sun = eph['sun']
earth = eph['earth']

location = earth + wgs84.latlon(68.35594288, 19.04520892)
astrometric = location.at(t).observe(sun)
alt, az, _ = astrometric.apparent().altaz()

print("Elevation:", alt.degrees)
print("Azimuth:", az.degrees)


[#################################] 100% de421.bsp


Elevation: 36.604571444148334
Azimuth: 162.93007232057565


In [40]:
import pandas as pd
from skyfield.api import load, wgs84
from datetime import datetime, timezone
from scipy.optimize import minimize_scalar

# Inputs
latitude = 68.35594288
longitude = 19.04520892
date = pd.Timestamp('1988-08-09', tz='UTC')
target_elevation = 36.63043526
target_azimuth = 162.30177663

# Load ephemeris data and time scale
eph = load('de421.bsp')
sun = eph['sun']
earth = eph['earth']
ts = load.timescale()

# Objective function: seconds since midnight
def objective(seconds_since_midnight):
    t = ts.utc(1988, 8, 9, 0, 0, seconds_since_midnight)
    location = earth + wgs84.latlon(latitude, longitude)
    astrometric = location.at(t).observe(sun)
    alt, az, _ = astrometric.apparent().altaz()
    return (alt.degrees - target_elevation)**2 + (az.degrees - target_azimuth)**2

# Optimize over a 24-hour day (in seconds)
res = minimize_scalar(objective, bounds=(0, 86400), method='bounded', options={'xatol': 0.01})
best_time_seconds = res.x

# Final timestamp
hours = int(best_time_seconds // 3600)
minutes = int((best_time_seconds % 3600) // 60)
seconds = best_time_seconds % 60
final_time = datetime(1988, 8, 9, hours, minutes, int(seconds), int((seconds % 1) * 1e6), tzinfo=timezone.utc)

print("Estimated time of day (UTC):", final_time.time())
print("Full datetime (UTC):", final_time)


Estimated time of day (UTC): 09:50:29.490115
Full datetime (UTC): 1988-08-09 09:50:29.490115+00:00
